# Проверка качества данных: Источник H2O (Факторы проблемности)

**Таблица:** `sandbox_ai.tmp_h20_fp_svy`  
**Формат источника:** xlsx  
**Содержание:** данные о факторах проблемности клиентов банка

## 0. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# === НАСТРОЙКИ ===
FILE_PATH = "../sources/data_h20.xlsx"

# Допустимые перечни — дополните под реальные данные
VALID_LEGAL_FORMS = {"ООО", "АО", "ПАО", "НАО", "ЗАО", "ИП", "ГУП", "МУП", "КФХ", "АНО", "НКО", "ТСЖ", "ПК", "КТ", "ОДО", "ФГУП", "ОП"}
VALID_SEGMENTS = {"ДМкб", "ДМБ", "ДМ", "ДСБ", "ДКБ", "ДРПА", "Не подлежит сегментации"}
VALID_MON_TYPES = {"Стандартный", "Упрощенный", "Индивидуальный"}
IP_LEGAL_FORMS = {"ИП", "КФХ"}
DATE_FORMAT = "%d.%m.%Y"
DATE_MIN = datetime(2023, 1, 1)
DATE_MAX = datetime(2025, 12, 31)

In [ ]:
# Загрузка xlsx — все колонки как строки, чтобы не потерять ведущие нули в ИНН
df = pd.read_excel(FILE_PATH, dtype=str)
df.columns = df.columns.str.strip()

print(f"Загружено строк: {len(df):,}, колонок: {len(df.columns)}")
print(f"Колонки: {list(df.columns)}")
df.head()

## 1. Пропуски (NULL / пустые значения)

In [ ]:
# Считаем пропуски по каждой колонке (NaN + пустые строки)
nulls = df.isna().sum() + (df.astype(str).apply(lambda c: c.str.strip() == '')).sum()
pct = (nulls / len(df) * 100).round(1)
null_report = pd.DataFrame({"Пропусков": nulls, "%": pct})
null_report["Статус"] = null_report["Пропусков"].apply(lambda x: "[!]" if x > 0 else "OK")
display(null_report)

# Критичные поля
for col in ["inn", "h20_fp_id", "fp_start_date", "ref_book_fp_id"]:
    if col in df.columns and nulls.get(col, 0) > 0:
        print(f"\n!!! КРИТИЧНО: '{col}' — {nulls[col]} пропусков. Это обязательное поле!")

## 2. Дубликаты

In [ ]:
# Полные дубликаты строк
full_dupes = df.duplicated().sum()
print(f"Полные дубликаты строк: {full_dupes}")

# Дубликаты h20_fp_id (должен быть уникальным)
if "h20_fp_id" in df.columns:
    h20_clean = df[df["h20_fp_id"].notna() & (df["h20_fp_id"].astype(str).str.strip() != "")]
    h20_dupes = h20_clean["h20_fp_id"].duplicated().sum()
    print(f"Дубликаты h20_fp_id: {h20_dupes}")

# Дубликаты crm_fp_id
if "crm_fp_id" in df.columns:
    crm_clean = df[df["crm_fp_id"].notna() & (df["crm_fp_id"].astype(str).str.strip() != "")]
    crm_dupes = crm_clean["crm_fp_id"].duplicated().sum()
    print(f"Дубликаты crm_fp_id: {crm_dupes}")

In [ ]:
# Анализ повторяющихся ИНН (у клиента может быть несколько ФП — это нормально)
if "inn" in df.columns:
    inn_counts = df["inn"].value_counts()
    print(f"Уникальных ИНН: {df['inn'].nunique():,}")
    print(f"ИНН с >1 записью: {(inn_counts > 1).sum():,}")
    if (inn_counts > 1).any():
        print("\nРаспределение количества ФП на клиента:")
        display(inn_counts.value_counts().sort_index().rename_axis("Кол-во ФП").reset_index(name="Клиентов"))
        print("\nТоп-5 клиентов по количеству ФП:")
        display(inn_counts.head(5).reset_index().rename(columns={"index": "ИНН", "inn": "Кол-во ФП"}))

# Дубликаты по паре inn + ref_book_fp_id
if "inn" in df.columns and "ref_book_fp_id" in df.columns:
    pair_dupes = df.duplicated(subset=["inn", "ref_book_fp_id"]).sum()
    print(f"\nДубликаты inn + ref_book_fp_id: {pair_dupes}")

# Одному ИНН — один cdi_id
if "inn" in df.columns and "cdi_id" in df.columns:
    inn_cdi = df.groupby("inn")["cdi_id"].nunique()
    mismatch = inn_cdi[inn_cdi > 1]
    print(f"ИНН с несколькими cdi_id: {len(mismatch)}")
    if len(mismatch) > 0:
        display(mismatch.head())

## 3. Формат и валидность значений

In [ ]:
# --- ИНН: только цифры, длина 10 или 12 ---
if "inn" in df.columns:
    inn_col = df["inn"].dropna().astype(str).str.strip()
    inn_col = inn_col[inn_col != ""]
    pat_digits = r'^\d+$'
    bad_chars = (~inn_col.str.match(pat_digits)).sum()
    bad_len = (~inn_col.str.len().isin([10, 12])).sum()
    len_dist = dict(inn_col.str.len().value_counts().sort_index())
    print("ИНН:")
    print(f"  Не только цифры: {bad_chars}")
    print(f"  Длина не 10 и не 12: {bad_len}")
    print(f"  Распределение по длине: {len_dist}")

In [ ]:
# --- Даты fp_start_date: формат, диапазон 2023-2025 ---
if "fp_start_date" in df.columns:
    raw = df["fp_start_date"].astype(str).str.strip()
    filled = raw[(raw != "") & (raw != "nan")]
    parsed = pd.to_datetime(filled, format=DATE_FORMAT, errors="coerce")
    bad_fmt = parsed.isna().sum()
    valid = parsed.dropna()
    before = (valid < DATE_MIN).sum()
    after = (valid > DATE_MAX).sum()

    print("fp_start_date:")
    print(f"  Некорректный формат: {bad_fmt}")
    print(f"  Ожидаемый период: {DATE_MIN.strftime(DATE_FORMAT)} — {DATE_MAX.strftime(DATE_FORMAT)}")
    print(f"  Ранее {DATE_MIN.strftime(DATE_FORMAT)}: {before}")
    print(f"  Позднее {DATE_MAX.strftime(DATE_FORMAT)}: {after}")
    print(f"  За пределами периода: {before + after}")
    if len(valid) > 0:
        print(f"  Фактический диапазон: {valid.min().strftime(DATE_FORMAT)} — {valid.max().strftime(DATE_FORMAT)}")

In [ ]:
# --- ref_book_fp_id: числовой, уникальные значения ---
if "ref_book_fp_id" in df.columns:
    ref_col = df["ref_book_fp_id"].dropna().astype(str).str.strip()
    ref_col = ref_col[ref_col != ""]
    pat_digits = r'^\d+$'
    non_numeric = (~ref_col.str.match(pat_digits)).sum()
    unique_refs = sorted(ref_col.unique(), key=lambda x: int(x) if x.isdigit() else 0)
    print(f"ref_book_fp_id:")
    print(f"  Нечисловые: {non_numeric}")
    print(f"  Уникальных значений: {len(unique_refs)}")
    if len(unique_refs) <= 100:
        print(f"  Значения: {unique_refs}")

## 4. Справочные значения (допустимые перечни)

In [ ]:
checks = [
    ("legal_form", "Организационно-правовая форма", VALID_LEGAL_FORMS),
    ("segment", "Сегмент бизнеса", VALID_SEGMENTS),
    ("mon_type", "Тип мониторинга", VALID_MON_TYPES),
]

for col, label, known in checks:
    if col not in df.columns:
        continue
    vals = set(df[col].dropna().astype(str).str.strip().unique()) - {""}
    unknown = vals - known
    print(f"{col} ({label}): {sorted(vals)}")
    if unknown:
        print(f"  [!] Неизвестные: {sorted(unknown)}")
    else:
        print(f"  [OK]")
    print()

# Региональные филиалы — выводим для ручной проверки
if "rf" in df.columns:
    print(f"rf — уникальных филиалов: {df['rf'].nunique()}")
    display(df["rf"].value_counts().rename_axis("Филиал").reset_index(name="Кол-во"))

## 5. Логическая согласованность

In [ ]:
# --- legal_form vs client_name ---
if "legal_form" in df.columns and "client_name" in df.columns:
    lf = df["legal_form"].astype(str).str.strip().str.upper()
    cn = df["client_name"].astype(str).str.strip().str.upper()
    starts = pd.Series(
        [c.startswith(l) if l and c else True for l, c in zip(lf, cn)],
        index=df.index,
    )
    mismatches = (~starts).sum()
    print(f"legal_form не совпадает с началом client_name: {mismatches}")
    if mismatches > 0:
        display(df[~starts][["legal_form", "client_name"]].head())

# --- legal_form vs длина ИНН (ИП=12, юрлицо=10) ---
if "legal_form" in df.columns and "inn" in df.columns:
    lf = df["legal_form"].astype(str).str.strip()
    inn_len = df["inn"].astype(str).str.strip().str.len()
    is_ip = lf.isin(IP_LEGAL_FORMS)
    bad = (is_ip & (inn_len != 12)) | (~is_ip & (inn_len != 10) & (lf != ""))
    print(f"\nlegal_form vs длина ИНН — несоответствий: {bad.sum()}")
    if bad.sum() > 0:
        display(df[bad][["inn", "legal_form"]].head())

# --- ИП/КФХ в сегменте крупного/среднего бизнеса ---
if "segment" in df.columns and "legal_form" in df.columns:
    suspicious = df[
        df["legal_form"].isin(["ИП", "КФХ"]) &
        df["segment"].isin(["ДКБ", "ДСБ"])
    ]
    print(f"\nИП/КФХ в сегменте ДКБ/ДСБ: {len(suspicious)}")
    if len(suspicious) > 0:
        display(suspicious[["client_name", "legal_form", "segment"]].head())

## 6. Статистический профиль

In [ ]:
# Распределение по сегментам
if "segment" in df.columns:
    print("Распределение по сегментам:")
    display(df["segment"].value_counts().rename_axis("Сегмент").reset_index(name="Кол-во"))

# Распределение по типам мониторинга
if "mon_type" in df.columns:
    print("\nРаспределение по типам мониторинга:")
    display(df["mon_type"].fillna("(пусто)").replace("", "(пусто)").value_counts()
            .rename_axis("Тип").reset_index(name="Кол-во"))

In [ ]:
# Распределение дат выявления ФП по годам
if "fp_start_date" in df.columns:
    dates = pd.to_datetime(df["fp_start_date"], format=DATE_FORMAT, errors="coerce")
    print("Распределение по годам:")
    display(dates.dt.year.value_counts().sort_index().rename_axis("Год").reset_index(name="Кол-во ФП"))

    month_dist = dates.dt.to_period("M").value_counts().sort_index()
    print(f"\nПиковый месяц: {month_dist.idxmax()} ({month_dist.max()} ФП)")

    day_dist = dates.value_counts()
    if day_dist.iloc[0] > 5:
        print("\nДаты с аномально большим числом ФП:")
        display(day_dist[day_dist > 3].rename_axis("Дата").reset_index(name="Кол-во ФП"))

# Топ-10 факторов проблемности
if "ref_book_fp_id" in df.columns:
    print("\nТоп-10 факторов проблемности:")
    display(df["ref_book_fp_id"].value_counts().head(10)
            .rename_axis("ref_book_fp_id").reset_index(name="Кол-во"))